Описание исследуемых датафреймов:

1.
olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей

customer_id — позаказный идентификатор пользователя

customer_unique_id —  уникальный идентификатор пользователя  (аналог номера паспорта)

customer_zip_code_prefix —  почтовый индекс пользователя

customer_city —  город доставки пользователя

customer_state —  штат доставки пользователя

2.
olist_orders_dataset.csv —  таблица заказов

order_id —  уникальный идентификатор заказа (номер чека)

customer_id —  позаказный идентификатор пользователя

order_status —  статус заказа

order_purchase_timestamp —  время создания заказа

order_approved_at —  время подтверждения оплаты заказа

order_delivered_carrier_date —  время передачи заказа в логистическую службу

order_delivered_customer_date —  время доставки заказа

order_estimated_delivery_date —  обещанная дата доставки

3.
olist_order_items_dataset.csv —  товарные позиции, входящие в заказы

order_id —  уникальный идентификатор заказа (номер чека)

order_item_id —  идентификатор товара внутри одного заказа

product_id —  ид товара (аналог штрихкода)

seller_id — ид производителя товара

shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике

price —  цена за единицу товара

freight_value —  вес товара

Уникальные статусы заказов в таблице olist_orders_dataset:

created —  создан; 
approved —  подтверждён; 
invoiced —  выставлен счёт; 
processing —  в процессе сборки заказа; 
shipped —  отгружен со склада; 
delivered —  доставлен пользователю; 
unavailable —  недоступен; 
canceled —  отменён

In [2]:
# Блок импорта библиотек
import pandas as pd
import requests
from urllib.parse import urlencode
import numpy as np
import plotly.express as px
from datetime import timedelta
import matplotlib.pyplot as plt
from datetime import datetime

Получим прямые ссылки на все датафреймы

olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей

olist_orders_dataset.csv —  таблица заказов

olist_order_items_dataset.csv — товарные позиции, входящие в заказы

In [4]:
# olist_customers_datase.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_1 = 'https://disk.yandex.ru/d/FUi5uSd6BfG_ig' 
# Получаем загрузочную ссылку
final_url_1 = base_url + urlencode(dict(public_key=public_key_1))
response = requests.get(final_url_1)
download_url_1 = response.json()['href']
olist_customers_dataset = pd.read_csv(download_url_1, sep=',')

# olist_orders_dataset.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_2 = 'https://disk.yandex.ru/d/t9Li4JOfgxuUrg' 
# Получаем загрузочную ссылку
final_url_2 = base_url + urlencode(dict(public_key=public_key_2))
response = requests.get(final_url_2)
download_url_2 = response.json()['href']
olist_orders_dataset = pd.read_csv(download_url_2, sep=',')

# olist_order_items_dataset.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_3 = 'https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw' 
# Получаем загрузочную ссылку
final_url_3 = base_url + urlencode(dict(public_key=public_key_3))
response = requests.get(final_url_3)
download_url_3 = response.json()['href']
olist_order_items_dataset = pd.read_csv(download_url_3, sep=',')

Посмотрим на все данные - красивое :)

In [5]:
olist_customers_dataset.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [6]:
olist_customers_dataset.shape

(99441, 5)

In [7]:
olist_orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [8]:
olist_orders_dataset.shape

(99441, 8)

In [9]:
olist_order_items_dataset.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [10]:
olist_order_items_dataset.shape

(112650, 7)

Проведем предварительные исследования данных. Ответим на ряд собственных вопросов.
В условии сказано: customer_id — позаказный идентификатор пользователя. Неясно, что значит "показаный". 
Будем считать, что это публичная информация, которая может появиться в каких-то документах (например, в чеке), 
а customer_unique_id - нет, и ее показывать нельзя. 
Посмотрим, как между собой соотносятся customer_id и customer_unique_i

In [11]:
olist_customers_dataset.nunique()

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

Видим, что их количество не совпадает, значит, предположение что одному customer_id соответствует один customer_unique_id
уже неверно. Т.к. customer_id больше, вероятно, у одного customer_unique_id может быть несколько customer_id, 
либо в строках есть нулевые значения. Проверим.

In [12]:
df = olist_customers_dataset.dropna()

In [13]:
df.nunique()

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

Нулеывых значений нет. Посмотрим, сколько уникальных сочетаний customer_id и customer_unique_id:

In [14]:
(olist_customers_dataset.customer_id.astype(str) + '_' +\
 olist_customers_dataset.customer_unique_id.astype(str)).nunique()

99441

столько же, сколько значений customer_id, и столько же, сколько строк в таблице. Т.е. повторений нет.
Вывод: у одного customer_unique_id может быть несколько customer_id. Проверим это дополнительно. 

In [15]:
olist_customers_dataset.loc[olist_customers_dataset\
                            .duplicated(subset=['customer_unique_id'])]\
 .sort_values('customer_unique_id', ascending=False).head(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
64323,4b231c90751c27521f7ee27ed2dc3b8f,ffe254cc039740e17dd15a5305035928,37640,extrema,MG
75057,1ae563fdfa500d150be6578066d83998,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,barra bonita,SP
79859,d064be88116eb8b958727aec4cf56a59,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,barra bonita,SP
92931,62a2c35611e8757e40fbbcecb3c9b2bd,ff8892f7c26aa0446da53d01b18df463,45972,ibiraja,BA
88272,09c682ef897935e677a0996cfe2008f3,ff44401d0d8f5b9c54a47374eb48c1b8,13070,campinas,SP
98551,75c6e03e7206bd1dd59f427c9f75142a,ff36be26206fffe1eb37afd54c70e18b,13057,campinas,SP
26608,3483b2a9e678865616e7d1b69b28dd4c,ff36be26206fffe1eb37afd54c70e18b,13057,campinas,SP
66296,4b7decb9b58e2569548b8b4c8e20e8d7,ff22e30958c13ffe219db7d711e8f564,2989,sao paulo,SP
69293,a1902b753d5eeb9831f1d3f89a13fba8,ff03923ad1eb9e32304deb7f9b2a45c9,18703,avare,SP
42097,4a4df909c2ef2c499e5be7d87d6349b6,fed519569d16e690df6f89cb99d4e682,20540,rio de janeiro,RJ


In [16]:
olist_customers_dataset.query('customer_unique_id == "ffe254cc039740e17dd15a5305035928"')

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
12133,0088395699ea0fcd459bfbef084997db,ffe254cc039740e17dd15a5305035928,37640,extrema,MG
64323,4b231c90751c27521f7ee27ed2dc3b8f,ffe254cc039740e17dd15a5305035928,37640,extrema,MG


Так и есть. Это значит, что за несколькими customer_id стоит один и тот же пользователь. И их количество 
нужно считать через customer_unique_id.

Теперь определимся с термином 'покупка'. Может ли быть отменен заказ, если он оплачен 
(если есть знаение в order_approved_at)?

In [17]:
olist_orders_dataset.query('order_status == "canceled"')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaN,NaN,2018-08-14 00:00:00
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaN,2018-02-22 00:00:00
1058,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2018-01-25 13:34:24,2018-01-25 13:50:20,2018-01-26 21:42:18,NaN,2018-02-23 00:00:00
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,NaN,NaN,NaN,2018-09-12 00:00:00
1801,ed3efbd3a87bea76c2812c66a0b32219,191984a8ba4cbb2145acb4fe35b69664,canceled,2018-09-20 13:54:16,NaN,NaN,NaN,2018-10-17 00:00:00
...,...,...,...,...,...,...,...,...
98791,b159d0ce7cd881052da94fa165617b05,e0c3bc5ce0836b975d6b2a8ce7bb0e3e,canceled,2017-03-11 19:51:36,2017-03-11 19:51:36,NaN,NaN,2017-03-30 00:00:00
98909,e49e7ce1471b4693482d40c2bd3ad196,e4e7ab3f449aeb401f0216f86c2104db,canceled,2018-08-07 11:16:28,NaN,NaN,NaN,2018-08-10 00:00:00
99143,6560fb10610771449cb0463c5ba12199,0d07d0a588caf93cc66b7a8aff86d2fe,canceled,2017-10-01 22:26:25,2017-10-01 22:35:22,NaN,NaN,2017-10-27 00:00:00
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00


In [18]:
olist_orders_dataset.query('order_status == "canceled"').dropna()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
2921,1950d777989f6a877539f53795b4c3c3,1bccb206de9f0f25adc6871a1bcf77b2,canceled,2018-02-19 19:48:52,2018-02-19 20:56:05,2018-02-20 19:57:13,2018-03-21 22:03:51,2018-03-09 00:00:00
8791,dabf2b0e35b423f94618bf965fcb7514,5cdec0bb8cbdf53ffc8fdc212cd247c6,canceled,2016-10-09 00:56:52,2016-10-09 13:36:58,2016-10-13 13:36:59,2016-10-16 14:36:59,2016-11-30 00:00:00
58266,770d331c84e5b214bd9dc70a10b829d0,6c57e6119369185e575b36712766b0ef,canceled,2016-10-07 14:52:30,2016-10-07 15:07:10,2016-10-11 15:07:11,2016-10-14 15:07:11,2016-11-29 00:00:00
59332,8beb59392e21af5eb9547ae1a9938d06,bf609b5741f71697f65ce3852c5d2623,canceled,2016-10-08 20:17:50,2016-10-09 14:34:30,2016-10-14 22:45:26,2016-10-19 18:47:43,2016-11-30 00:00:00
92636,65d1e226dfaeb8cdc42f665422522d14,70fc57eeae292675927697fe03ad3ff5,canceled,2016-10-03 21:01:41,2016-10-04 10:18:57,2016-10-25 12:14:28,2016-11-08 10:58:34,2016-11-25 00:00:00
94399,2c45c33d2f9cb8ff8b1c86cc28c11c30,de4caa97afa80c8eeac2ff4c8da5b72e,canceled,2016-10-09 15:39:56,2016-10-10 10:40:49,2016-10-14 10:40:50,2016-11-09 14:53:50,2016-12-08 00:00:00


Видим, что может. Более того, может быть отменен, даже если есть order_delivered_customer_date — время доставки заказа.
Поэтому логично считать, то покупка совершена (завершена), только если статус заказа delivered.

Предварительные выводы:

1. У одного customer_unique_id может быть несколько customer_id.

2. Покупка считается совершенной, если статус заказа delivered.